<h1 style="text-align: center;">CQF - FINAL EXAM </h1>
<h1 style="text-align: center;">Portfolio Construction using Black-Litterman Model and Factors </h1>
<h3 style="text-align: center;">Michele Vannucci</h3>
<h3 style="text-align: center;">January 2025 Cohort</h3>

---

# Instructions
This notebook presents the core code structure accompanied by essential technical comments. For clarity and readability, all plots and printed outputs are omitted here, as well as theoretical explanations of the framework; they are included and discussed in detail in the accompanying PDF report.
Only technical aspects are reported in this document, such as function design, algorithms descriptions, data manipulation procedures, and output formats. The notebook is designed to run autonomously. Some of the required data files are retrieved directly through the code, while others were downloaded manually. All manually obtained files will be included in the same zip archive as this notebook and are assumed to be stored in a `Data/` directory located in the notebook’s root folder.
The Python libraries used are listed together with their versions, and the entire project was executed in a virtual environment running Python 3.10.16.

**Notation**
- Dates: reported as `YY-MM-DD`.
- Data frequency: computations use weekly returns; all results are reported at an annualized frequency.
- Vectors and matrices are denoted in boldface, while the transposition operator is indicated with a prime symbol ($'$). The identity matrix is written as $\mathbb{I}$

In [ ]:
# Standard library
import time

# Scientific computing
import numpy as np                          # v 2.1.3
import pandas as pd                         # v 2.2.3
import matplotlib.pyplot as plt             # v 3.10.1
import seaborn as sns                       # v 0.13.2

# Data retrieval
import yfinance as yf                       # v 0.2.61

# Statistical modeling
import statsmodels.api as sm                # v 0.14.4
from scipy.stats import norm

# Optimization
from scipy.optimize import minimize, \
                           minimize_scalar, \
                           LinearConstraint # v 1.15.2
from sklearn.covariance import LedoitWolf   # v 1.6.1
from cvxopt import matrix, solvers          # v 1.3.2

# Data retrieval and preprocessing
The workflow follows this structure:
1. **Data acquisition** – downloading required datasets, either manually or programmatically.
2. **Index alignment** – ensuring consistent time periods across all dataframes and defining start and end dates.
3. **Resampling** – converting all series to weekly frequency, using Friday as the reference date.
4. **Computation of excess returns** – calculating returns in excess of the risk-free rate.
  
The following datasets have beeen manually downloaded, as they are not retrieved automatically by the notebook:
- **Fama-French 5 factors**   
Available at https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html  
Section U.S. *Research Returns Data* → *Fama/French 5 Factors (2x3) [Daily]*  
Saved as: `FF5_Factors_daily.csv`
- **Momentum Factor**  
Available at the same link, under *Sorts involving Prior Returns* → *Momentum Factor (Mom) [Daily]*  
Saved as: `FF_Mom_daily.csv`
- **Betting Against Beta (BAB)**  
Downloaded from https://www.aqr.com/Insights/Datasets/Betting-Against-Beta-Equity-Factors-Daily (daily data)  
The file, originally provided in `.xlsx` format, is converted to `.csv` and contains multiple countries. Only the USA column is used.  
Saved as: `BAB_AQR_daily.csv`
- **3-Month Treasury Bill Rate**  
Available at https://fred.stlouisfed.org/series/TB3MS (monthly data)  
Saved as: `TB3MS.csv`

## Data acquisition

The list below contains all tickers selected for the analysis. `factor_proxy_tickers` stores the ETFs considered as potential proxies for the factors, while `asset_tickers` contains the remaining portfolio assets. `spy_ticker` is used as the benchmark. Daily adjusted close prices for all ETF tickers are saved as `.csv` files in the `Data/` folder for offline processing. All available historical data are retained. 

In [ ]:
spy_ticker = "SPY"   

factor_proxy_tickers = ["IJR","VB","SLYV","SCHA","IWM",       # SMB
                        "VLUE","SPVU","VTV","IVE","IVLU",     # HML
                        "SPMO","QMOM","MTUM",                 # MOM
                        "QUAL",                               # RMW
                        "SYLD",                               # CMA
                        "SPLV", "SPHB"                        # BAB
                        ]     

asset_tickers = [
                 "QQQ",                           # growth
                 "SPHQ",                          # quality  
                 "XLE","XLP","XLV","XLF","XLU",   # sector rotation
                 "TLT","IEF",                     # rates and term structure
                 "LQD","HYG",                     # credit spread risk
                 "TIP",                           # inflation beta
                 "VNQ",                           # real estate
                 "GLD",                           # gold
                 "DBC",                           # commodities
                 "VIXM",                          # volatility
                 ]    

In [ ]:
data_requests = {
    "benchmark": {
        "tickers": spy_ticker, 
        "filename": "Data/spy_daily.csv"
    },
    "factors": {
        "tickers": factor_proxy_tickers, 
        "filename": "Data/factor_proxy_daily.csv"
    },
    "assets":{
        "tickers": asset_tickers,
        "filename": "Data/asset_prices_daily.csv"
    }
}

for name, spec in data_requests.items():
    tickers = spec["tickers"]
    tickers = [tickers] if isinstance(tickers, str) else tickers

    df = yf.download(
        tickers,
        interval="1d",
        auto_adjust=True,
        progress=False
    )["Close"]

    df.to_csv(spec["filename"])
    print(f"Saved {name} data to {spec['filename']}")

## Index and dates alignment

In [ ]:
proxy_daily_close = pd.read_csv("Data/factor_proxy_daily.csv", index_col="Date", parse_dates=True)
FF5_daily_ret     = pd.read_csv("Data/FF5_Factors_daily.csv",  index_col="Date", parse_dates=True) 
Mom_daily_ret     = pd.read_csv("Data/FF_Mom_daily.csv",       index_col="Date", parse_dates=True) 
BAB_daily_ret     = pd.read_csv("Data/BAB_AQR_daily.csv",      index_col="Date", parse_dates=True) 
rf3m_monthly      = pd.read_csv('Data/TB3MS.csv',              index_col="Date", parse_dates=True) 
spy_daily_close   = pd.read_csv('Data/spy_daily.csv',          index_col="Date", parse_dates=True) 
asset_daily_close = pd.read_csv("Data/asset_prices_daily.csv", index_col="Date", parse_dates=True)

It is essential that all data frames share the same time window and have a consistent index. The initial and final dates are identified by checking the first and last valid observations across all series. The youngest (latest-starting) ETF determines the sample start, while the series with the earliest termination determines the sample end. Daily prices are later converted to weekly frequency by taking the Friday close, so the time grid is aligned to Fridays. The sample ends on 2025-04-25, the last Friday on or before the earliest final date among all series, corresponding to the Betting Against Beta factor. Because the inaugural week (Wednesday 2015-12-02 to Friday 2015-12-04) lacks a prior Friday close, its return is undefined and automatically discarded, making the week ending 2015-12-11 the first valid observation. This truncated two-day span at the start of the sample is negligible.

In [ ]:
df_all = pd.concat([proxy_daily_close, FF5_daily_ret, Mom_daily_ret, BAB_daily_ret], axis=1)

# First and last available dates for each column
first_dates = df_all.apply(lambda col: col.first_valid_index())
last_dates  = df_all.apply(lambda col: col.last_valid_index())

# Youngest (latest starting) series
youngest_ticker = first_dates.idxmax()
youngest_date   = first_dates.max()

# Oldest (earliest ending) series
oldest_end_ticker = last_dates.idxmin()
oldest_end_date   = last_dates.min()

start_date = pd.Timestamp(year=youngest_date.year,
                          month=youngest_date.month,
                          day=1)

# Find last available Friday 
end_date = oldest_end_date - pd.Timedelta(days=(oldest_end_date.weekday() - 4) % 7)

print(f"Start date:   {start_date}")
print(f"End date:     {end_date}")
print(f"Youngest ETF: {youngest_ticker} ({youngest_date})")
print(f"Earliest end: {oldest_end_ticker} ({end_date})")

In [ ]:
proxy_daily_close = proxy_daily_close.loc[start_date : end_date]
FF5_daily_ret     = FF5_daily_ret    .loc[start_date : end_date]
Mom_daily_ret     = Mom_daily_ret    .loc[start_date : end_date]
BAB_daily_ret     = BAB_daily_ret    .loc[start_date : end_date]
rf3m_monthly      = rf3m_monthly     .loc[start_date : end_date]
spy_daily_close   = spy_daily_close  .loc[start_date : end_date]
asset_daily_close = asset_daily_close.loc[start_date : end_date]

## Resampling

Factors and Treasurey Bill data are expressed as percent returns, so they need to be scaled back to decimals

In [ ]:
FF5_daily_ret = FF5_daily_ret/100.0
Mom_daily_ret = Mom_daily_ret/100.0
BAB_daily_ret = BAB_daily_ret/100.0
rf3m_monthly  = rf3m_monthly/100.0

The Fama-French file reports the market excess return as `Mkt-RF`, where the risk-free leg is the one-month Treasury bill (`RF`). Because this study adopts the three-month bill as its risk-free asset, `Mkt-RF` must first be turned back into a total market return by adding `RF`; the excess return relative to the three-month bill will then obtained by subtracting the three-month yield.  

In [ ]:
FF5_daily_ret['MKT'] = FF5_daily_ret['Mkt-RF'] + FF5_daily_ret['RF']
FF5_daily_ret.drop(['Mkt-RF','RF'], axis=1, inplace=True)

Each dataset is converted to a coherent Friday-to-Friday panel before excess returns are formed. Weeks in which U.S. markets are closed on the Friday contain no daily observation, but the for the weekly resampling Thursday data is used, thanks to `.last()`. An explicit example is Good Friday, 2022-04-15, which is filled using data from 2022-04-14 data.
- Daily closing prices are resampled to Friday observations with `resample("W-FRI").last()`, after which one-week simple returns are obtained via `pct_change()`: $$r^{\text{week}}_t = \dfrac{P_t}{P_{t-1}} - 1$$
- Daily Fama-French factor values are already expressed as daily simple returns, so the corresponding weekly return is derived by compounding within each week:
  $$r^{\text{week}}_t \;=\; \prod_{d\in\text{week}}\bigl(1 + r_d\bigr) \;-\; 1$$
- The three–month Treasury-bill series is supplied as a monthly annualized yield. First, each observation is converted into a one-week simple return: $$r^{\text{week}}_t \;=\; \bigl(1 + y^{\text{ann}}_t\bigr)^{1/52} - 1$$ 
Then it is resampled so that each Friday inside a given month receives the most recently published monthly yield. Then the series is extended up to the end of April using `.reindex(full_fridays)`, where `full_fridays` is the grid of all Fridays during the time window considered; these slots are finally forward filled.

In [ ]:
proxy_weekly_ret = proxy_daily_close.resample("W-FRI").last().pct_change()
spy_weekly_ret   = spy_daily_close  .resample("W-FRI").last().pct_change()
asset_weekly_ret = asset_daily_close.resample("W-FRI").last().pct_change()

FF5_weekly_ret = (1 + FF5_daily_ret).resample("W-FRI").prod() - 1
Mom_weekly_ret = (1 + Mom_daily_ret).resample("W-FRI").prod() - 1
BAB_weekly_ret = (1 + BAB_daily_ret).resample("W-FRI").prod() - 1

WEEKS_PER_YEAR = 52
full_fridays = pd.date_range(start_date, end_date, freq="W-FRI")
rf3m_weekly = (1 + rf3m_monthly).pow(1/WEEKS_PER_YEAR) - 1      

rf3m_weekly = (rf3m_weekly
             .resample("W-FRI").ffill()
             .reindex(full_fridays)
             .ffill()
            )  

In [ ]:
# grouping together all factor data
factors_weekly_ret = pd.concat([FF5_weekly_ret, Mom_weekly_ret, BAB_weekly_ret], axis=1)

In [ ]:
print('ETF proxy:    ', proxy_weekly_ret.shape)
print('FF factors:   ', factors_weekly_ret.shape)
print('TB3MS:        ', rf3m_weekly.shape)
print('SPY benchmark:', spy_weekly_ret.shape)

Excess returns are obtained by subtracting the three-month Treasury-bill rate from the corresponding simple returns. Because the Fama–French factor series are already defined net of the risk-free rate, this adjustment is applied solely to the market return and to each ETF proxy; the factor series themselves remain unchanged.

In [ ]:
factors_exc = factors_weekly_ret.copy()
factors_exc['MKT'] = factors_weekly_ret['MKT'].subtract(rf3m_weekly['TB3MS'], axis=0)

proxy_exc = proxy_weekly_ret.subtract(rf3m_weekly['TB3MS'], axis=0)
spy_exc   = spy_weekly_ret  .subtract(rf3m_weekly['TB3MS'], axis=0)
asset_exc = asset_weekly_ret.subtract(rf3m_weekly['TB3MS'], axis=0)

proxy_exc   = proxy_exc.dropna()
spy_exc     = spy_exc.dropna()
factors_exc = factors_exc.dropna()
asset_exc   = asset_exc.dropna()

Building the *Bet-Against-Beta* custom factor proxy as the difference of excess returns of $\text{SPLV}$ and $\text{SPHB}$: $$\text{BAB} = r_{\text{SPLV}} - r_{\text{SPHB}}$$

In [ ]:
proxy_exc['BAB_p'] = proxy_exc['SPLV'] - proxy_exc['SPHB'] # _p stands for 'proxy' to avoid naming confusion
proxy_exc.drop(['SPLV','SPHB'], axis=1, inplace=True)

---

# Factor selection
The objective is to identify suitable ETF proxies for the Fama–French factors. To this end, an OLS regression including all factors is estimated for each candidate. The best proxies are selected according to the following guidelines:
- **Factor sensitivity**: $\beta > 0.8$ ensures a strong sensitivity to the intended factor.
- **Cross-factor neutrality**: $|\beta| < 0.3$ limits unintended exposures to other factors.
- **Market exposure**: $\beta \in [0.8,1.2]$ confirms appropriate sensitivity to the benchmark.
- **No unexplained drift**: $\alpha \approx 0$ indicates the absence of persistent unexplained performance.
- **Model fit**: $R^{2} > 0.8$ requires that at least 80\% of return variability is explained by the factor model.

These thresholds are indicative rather than absolute; in practice, some may be relaxed if justified.

The weekly excess‐return of each ETF is regressed on the set of the 5 Fama-French factors plus the Momentum and Betting Against Beta: $\text{factors} = \{\text{MKT, SMB, HML, RMW, CMA, MOM, BAB}\}$ by ordinary least squares(OLS): $$r_t^{\text{ETF}}=\alpha + \sum_{k \,\in\, \text{factors}} \beta_{k_t} \cdot \,k_t + \varepsilon_t$$
Here $\alpha$ captures any systematic drift or tracking-error that is not explained by the factors, while $\varepsilon_t$ represents the remaining idiosyncratic shock and is assumed to have zero mean. 

To obtain heteroskedasticity- and autocorrelation-consistent inference, Newey–West (HAC) standard errors are computed with four lags. Four weekly lags capture one calendar month of serial correlation, which is sufficient for assets whose memory rarely exceeds that horizon. The HAC adjustment leaves the point estimates $\alpha$ and $\beta_k$ unchanged but replaces the usual OLS standard errors with robust ones, allowing valid t-statistics.

In [ ]:
def run_proxy_test(
        etf, target_factor,
        proxy_exc, factors_exc,
        nw_lags=4
):
    """
    OLS + Newey-West factor regression.
    proxy_exc: DataFrame of ETF excess returns (proxies)
    factors_exc: DataFrame of factor excess returns
    """
    
    factors = ['MKT','SMB','HML','MOM','RMW','CMA','BAB']
    
    def _nw_fit(y, X):
        ols = sm.OLS(y, X).fit()
        nw  = ols.get_robustcov_results(cov_type="HAC", maxlags=nw_lags)
        p   = pd.Series(nw.params,  index=ols.params.index)
        t   = pd.Series(nw.tvalues, index=ols.params.index)
        return p, t, ols.rsquared
    
    # Align and merge the ETF and factors
    y = proxy_exc[etf]
    X = factors_exc[factors]
    y, X = y.align(X, join="inner", axis=0)
    X = sm.add_constant(X)
    
    beta, tstat, r2 = _nw_fit(y, X)
    
    cross = {
        f: beta[f] for f in factors if f != target_factor
    }

    # Results
    print(f"\n=== {etf} vs {target_factor} ===")
    print(f"β_{target_factor}: {beta[target_factor]:.2f}, "
          f"NW t-stat: {tstat[target_factor]:.2f}")
    if cross:
        print("Factor loadings:",
              {k: f"{v:.2f}" for k, v in cross.items()})
    print(f"α (weekly): {beta['const']:.4f} "
          f"→ ≈ {beta['const']*WEEKS_PER_YEAR:.2%} p.a., "
          f"t-stat: {tstat['const']:.2f}")
    print(f"R² = {r2:.2f}")
    return beta, tstat, r2

The regression is performed only for meaningful proxy–factor pairs, rather than for every possible combination.

In [ ]:
proxy_groups = {
    "SMB": ["IJR","IWM","VB","SCHA","SLYV"],
    "HML": ["VLUE","IVLU","VTV","IVE","SPVU"],
    "MOM": ["MTUM","QMOM","SPMO"],
    "RMW": ["QUAL"],
    "CMA": ["SYLD"],                
    "BAB": ["BAB_p"]           
}

for factor, etfs in proxy_groups.items():
    for etf in etfs:
        run_proxy_test(
            etf=etf,
            target_factor=factor,
            proxy_exc=proxy_exc,
            factors_exc=factors_exc,
            nw_lags=4
        )

A detailed analysis of the regression results is provided in the report. The final set of ETFs selected as factor proxies are: $\text{IWM}$ for $\text{SMB}$, $\text{SPVU}$ for $\text{HML}$, $\text{MTUM}$ for $\text{MOM}$, and $\text{BAB_p}$ for $\text{BAB}$.

Checking if $\alpha_{\text{IWM}}$ is economically insignificant with respect to annual volatility.

In [ ]:
iwm_vol_annual = proxy_exc['IWM'].std(ddof=1) * np.sqrt(52)
print(f"IWM annual volatility: {(iwm_vol_annual*100).round(2)}% p.a.")

## P&L Analysis
Comparing the cumulative performance of the selected factor proxies against the benchmark ($\text{SPY}$). Weekly excess returns are aggregated, and cumulative returns are computed as: $$ \text{CumRet}_t = \prod_{s=1}^t (1+r_s)-1$$

In [ ]:
facts = ['IWM','SPVU','MTUM','BAB_p']
final_factors_exc = proxy_exc[facts]
final_factors_exc = final_factors_exc.dropna()

final_exc = pd.concat([final_factors_exc, spy_exc], axis=1)
cum_ret = (1 + final_exc).cumprod() - 1

fig, ax = plt.subplots(figsize=(10, 6))

cum_ret.plot(ax=ax, linewidth=1.4)                     
ax.set_title("Cumulative Return — Factors vs SPY", fontsize=14, weight='bold')
ax.set_ylabel("Cumulative return")
ax.set_xlabel("")                                     
ax.axhline(0, color="grey", lw=0.8, linestyle="--")   
ax.legend(ncol=3, frameon=False)
plt.tight_layout()
plt.show()

## Rolling quantities plots
Rolling $\alpha$ and $\beta$ estimates are computed over a two-year moving window. While $\beta$ is dimentionless, $\alpha$  represents a weekly return and is therefore annualized for interpretation. The definitions adopted are: $$\beta = \frac{\text{Cov}(r_{asset}, f)}{\text{Var}(f)}$$ $$\alpha = \bar{r}_{asset} - \beta \,\bar{f}$$ where bars denote sample means over the chosen time period. 

In [ ]:
selected_pairs = {
    "IWM   vs SMB":  ("IWM",  "SMB"),
    "SPVU  vs HML": ("SPVU", "HML"),
    "MTUM  vs MOM": ("MTUM", "MOM"),
    "BAB_p vs BAB":("BAB_p","BAB") 
}

window = 104  # ≈ 2 years

for title, (etf, factor) in selected_pairs.items():

    y = proxy_exc[etf]
    x = factors_exc[factor]
    data = pd.concat([y, x], axis=1).dropna()        
    y, x = data.iloc[:, 0], data.iloc[:, 1]          

    rolling_cov  = y.rolling(window).cov(x)
    rolling_var  = x.rolling(window).var()
    rolling_beta = rolling_cov / rolling_var

    mean_y  = y.rolling(window).mean()
    mean_x  = x.rolling(window).mean()
    rolling_alpha = (mean_y - rolling_beta * mean_x) * WEEKS_PER_YEAR 

    # plotting
    fig, ax1 = plt.subplots(figsize=(10, 4))
    ax1.plot(rolling_beta.index, rolling_beta, label="Beta", color="tab:blue")
    ax1.set_ylabel("β", color="tab:blue")
    ax1.tick_params(axis='y', labelcolor="tab:blue")

    ax2 = ax1.twinx()
    ax2.plot(rolling_alpha.index, rolling_alpha,
             label="Alpha (annualised)", color="tab:red", linestyle="--")
    ax2.set_ylabel("α  (annual %)", color="tab:red")
    ax2.tick_params(axis='y', labelcolor="tab:red")

    ax1.set_title(f"Rolling {window}-Week β and α: {title}")
    ax1.axhline(0, lw=0.7, ls=":", color="tab:blue") # zero-beta
    ax2.axhline(0, lw=0.7, ls=":", color="tab:red") # zero-alpha
    fig.tight_layout()
    plt.show()

## Performance Statistics 

For each asset or factor return series, the following annualized performance metrics are computed from weekly excess returns:

- **Annual Return**: Geometric mean return scaled to an annual horizon, $$\text{Ann. Ret} = \left[ \prod_{t=1}^{T} (1 + r_t) \right]^{\frac{52}{T}} - 1$$ 

- **Annual Volatility**: Standard deviation of weekly returns multiplied by the square root of the annualization factor, $$\text{Ann. Vol} = \sigma_{\text{weekly}} \times \sqrt{52}$$ using the sample standard deviation (`ddof=1`).

- **Sharpe Ratio**: Risk-adjusted return using the ratio of annual return to annual volatility, $$\text{Sharpe} = \frac{\text{Ann. Ret}}{\text{Ann. Vol}}$$ (since the returns are in excess form, no risk-free subtraction is required).

- **Maximum Drawdown**: Largest peak-to-trough decline in the cumulative return curve, $$\text{Max DD} = \min_{t} \left( \frac{V_t}{\max_{s \leq t} V_s} - 1 \right)$$ where $V_t = \text{CumRet}_t + 1$ is the cumulative return index.

These metrics summarise both absolute and risk-adjusted performance, allowing for a direct comparison of factor proxies and the benchmark over the sample period.

In [ ]:
results = {}

for col in final_exc.columns:
    r = final_exc[col]
    
    ann_ret = (1 + r).prod()**(WEEKS_PER_YEAR / len(r)) - 1
    ann_vol = r.std(ddof=1) * np.sqrt(WEEKS_PER_YEAR)  
    sharpe  = ann_ret / ann_vol
    dd_curve = (1 + r).cumprod()
    max_dd  = ((dd_curve / dd_curve.cummax()) - 1).min()

    results[col] = [ann_ret, ann_vol, sharpe, max_dd]

stats_df = pd.DataFrame.from_dict(
    results, 
    orient='index', 
    columns=['Annual Ret', 'Annual Vol', 'Sharpe', 'Max DD']
)

print(stats_df)

---

# Correlation Structure and Covariance Matrix

Once the asset universe is defined, the first step is to analyse the correlation and covariance matrices. This stage is crucial, as all subsequent results will be heavily influenced by these estimates. Moreover, a poorly specified covariance matrix can lead to unstable results and even cause algorithmic failures, particularly in operations requiring matrix inversion.

In [ ]:
asset_universe_exc = pd.concat([final_factors_exc, asset_exc], axis=1)
tickers = asset_universe_exc.columns

corr_matrix = asset_universe_exc.corr()
plt.figure(figsize=(20, 16))
plt.title("Feature Correlation Heatmap")
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0, linewidths=0.5)
plt.show()

Because the sample spans 2015 – 2025, it straddles the COVID-19 shock—a period that may have altered the dependence structure among ETFs.
Before estimating the full-sample covariance matrix, the data are therefore split at 28 Feb 2020. For each segment the pair-wise correlation matrix is calculated and its grand mean reported; the difference matrix $\Delta\boldsymbol{\rho}=\boldsymbol{\rho}_{post} - \boldsymbol{\rho}_{pre}$ is then visualised as a heat-map to highlight where correlations strengthened (red) or weakened (blue).

In [ ]:
cut = "2020-02-28"
pre  = asset_universe_exc.loc[:cut]
post = asset_universe_exc.loc[cut:]

corr_pre  = pre.corr()
corr_post = post.corr()

avg_pre  = corr_pre.values[np.triu_indices(20, k=1)].mean()
avg_post = corr_post.values[np.triu_indices(20, k=1)].mean()
print(f"Pre-COVID avg ρ = {avg_pre:.2f}, post-COVID avg ρ = {avg_post:.2f}")

In [ ]:
corr_diff = corr_post - corr_pre

# redefine scale limits for better image clarity
band = 0.65        
vmin, vmax = -band, band

plt.figure(figsize=(12, 10))
im = plt.imshow(corr_diff, cmap="coolwarm", vmin=vmin, vmax=vmax)
plt.colorbar(im, fraction=0.046, pad=0.04,
             label="Δ Correlation (Post - Pre)")
plt.xticks(ticks=np.arange(len(tickers)), labels=tickers, rotation=90)
plt.yticks(ticks=np.arange(len(tickers)), labels=tickers)
plt.title("Change in Pairwise Correlations\n(Post-COVID - Pre-COVID)")
plt.tight_layout()
plt.show()

An average correlation jump from 0.20 → 0.30 around COVID is non-trivial and a static $\boldsymbol{\Sigma}$ estimated over the full 2015–2025 span would blend two very different regimes. For this reason, an exponentially weighted covariance (EWMA) is employed.
The exponentially weighted mean is updated recursively, $$\boldsymbol{\mu}_t = \alpha \boldsymbol{r}_t + (1-\alpha)\boldsymbol{\mu}_{t-1}$$ and the corresponding covariance matrix is $$\boldsymbol{\Sigma}_t = \alpha (\boldsymbol{r}_t-\boldsymbol{\mu}_t)(\boldsymbol{r}_t-\boldsymbol{\mu}_t)'+(1-\alpha)\boldsymbol{\Sigma}_{t-1}$$  
The smoothing parameter $\alpha$ controls the emphasis placed on recent observations: values closer to 1 assign greater weight to the most recent regime, which is particularly relevant for capturing the structural break introduced by the COVID-19 period. In practice, it is often expressed in terms of the decay factor $\lambda = 1 - \alpha$, which specifies the weight assigned to the previous covariance estimate. Following the RiskMetrics (2001) guidelines, $\lambda=0.94$ is suggested for daily data. To adapt this to weekly data, the decay factor is rescaled as $\lambda_{\text{weekly}} = \lambda_{\text{daily}}^5$, assuming five trading days per week. The smoothing parameter is then: $$\alpha = 1 - \lambda_{weekly}$$

In [ ]:
alpha = 0.2661 # 1 - (0.94)^5
cov_ewma = asset_universe_exc.ewm(alpha=alpha, adjust=False).cov()
Sigma_df = cov_ewma.loc[end_date]
Sigma = Sigma_df.to_numpy()

Before proceeding, the condition number of the covariance matrix $\boldsymbol{\Sigma}$ is assessed by computing the ratio between its smallest and largest eigenvalues. If this ratio is exceedingly small (i.e., $\ll 10^{-5}$), then $\boldsymbol{\Sigma}$ is deemed nearly singular.

In [ ]:
eigs = np.linalg.eigvalsh(Sigma)    
print("Smallest eigenvalue:", eigs.min())
print("Largest eigenvalue:", eigs.max())
print("Ratio λ_min/λ_max:", eigs.min()/eigs.max())

The ratio between the largest and smallest eigenvalues is on the order of $10^{-7}$, indicating that the covariance matrix is numerically ill-conditioned.  
To regularise the estimate, a Ledoit–Wolf shrinkage estimator is applied instead: $$\boldsymbol{\Sigma}_{\text{LW}} = (1-\delta)\boldsymbol{\Sigma}+\delta \frac{\text{Tr}(\boldsymbol{\Sigma})}{N} \mathbb{I}_N$$ where $N$ is the dimensionality of the matrix, $\mathbb{I}_N$ is the ($N \times N$) identity matrix, $\text{Tr}(\cdot)$ denotes the trace operator and $\delta$ is the shrinkage intensity chosen to minimise the mean-squared error between $\boldsymbol{\Sigma}_{\text{LW}}$ and the (unknown) true covariance matrix.

In [ ]:
returns = asset_universe_exc.values    
lw = LedoitWolf(store_precision=True).fit(returns)
Sigma_lw  = lw.covariance_      

In [ ]:
eigs = np.linalg.eigvalsh(Sigma_lw)    
print("Smallest eigenvalue:", eigs.min())
print("Largest eigenvalue:", eigs.max())
print("Ratio λ_min/λ_max:", eigs.min()/eigs.max())

A clear improvement is achieved, with the eigenvalue ratio increasing to the order of $10^{-3}$. The shrunk covariance matrix will therefore be used in all subsequent operations.

---

# Black–Litterman Model

The implementation proceeds in four stages.
1. The equilibrium excess-return vector $\boldsymbol{\pi}$ is estimated after computing $\mathbf{w}_{\text{mkt}}$ and $\lambda_{\text{mkt}}$.
2. Definition of the views and their uncertainties. 
3. The view-uncertainty matrix $\boldsymbol{\Omega}$ is built. 
4. Posterior return and optimal weights 

A table precedes each stage to list the variables introduced up to that point and to record their dimensions, as matrix operations can become hard to follow.
Throughout, a column vector is written $(M\times1)$ and a row vector $(1\times M)$.
Consequently, a row–by–column product produces a scalar $(1\times1)$, whereas a column–by–row product produces an outer-product matrix $(M\times M)$.  
Variables appear in every table in the exact order in which they are generated by the accompanying code.

## Part I - Equilibrium Return

<div style="margin:20px 0; width:500px;">
  <table style="border-collapse:collapse; font-family:Arial, sans-serif; font-size:14px;">
    <colgroup>
      <col style="width:100px; text-align:center;">
      <col style="width:90px; text-align:center;">
      <col style="width:280px; text-align:left;">
    </colgroup>
    <thead>
      <tr style="background-color:#f2f2f2;">
        <th style="border:1px solid #ccc; padding:6px; text-align:center;">Variable</th>
        <th style="border:1px solid #ccc; padding:6px; text-align:center;">Shape</th>
        <th style="border:1px solid #ccc; padding:6px; text-align:left;">Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{w}_{\text{mkt}}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Market Weights</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\lambda_{\text{mkt}}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$1 \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Market Risk Aversion coefficient</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\boldsymbol{\Sigma}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times N$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Covariance Matrix (Ledoit–Wolf shrunk)</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\boldsymbol{\pi}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Implied Equilibrium Return Vector</td>
      </tr>
    </tbody>
  </table>
</div>


### Market Weights
Market weights are used to construct the prior return vector. Five asset classes are defined: Equities, Fixed Income, Real Assets, Factors, and Volatility. Each portfolio constituent is assigned to one of these categories, as asset classes typically have distinct allocation weights in global portfolios. The partition reported by State Street Investment Management in Global Market Portfolio (GMP) 2024 serves as the reference. Details on the GMP composition and weight computation are provided in the accompanying PDF report.  

For each asset in the portfolio, market capitalization is calculated as: $$\text{Market Cap} = \text{Shares Outstanding} \times \text{Price}$$ If this information is unavailable in Yahoo Finance, Assets Under Management (AUM) are used as an approximation.  

The Factors and Volatility categories are explicitly assigned zero weight, as these exposures are synthetic constructs rather than directly investable securities with a market capitalization. Finally, within each basket, individual asset weights are determined proportionally to their market capitalizations, and each basket is normalised to sum to 100%.

In [ ]:
asset_classes = {
    "Equities":     ["QQQ","SPHQ","XLE","XLF","XLP","XLU","XLV"],
    "Fixed Income": ["LQD","HYG","IEF","TLT","TIP"],
    "Real Assets":  ["GLD","VNQ","DBC"],
    "Factors":      ["IWM","SPVU","MTUM","BAB_p"],
    "Volatility":   ["VIXM"]
}

In [ ]:
# Market capitalizations expressed in Trillion USD
GMP_alloc = {
    "Equities": 78.0,
    "Fixed Income": 57.23,
    "Real Assets": 12.95
}

# computing asset classes weights
total_included = sum(GMP_alloc.values())
GMP_percent = {k: v / total_included for k, v in GMP_alloc.items()}

In [ ]:
exclude = {"IWM", "SPVU", "MTUM", "BAB_p", "VIXM"}
subset_tickers = [t for t in asset_universe_exc.columns if t not in exclude]

rows = []
for tic in subset_tickers:
    tk = yf.Ticker(tic)
    info = tk.info or {}
    
    # Try market cap from sharesOutstanding × price
    mc = None
    shares = info.get("sharesOutstanding")
    if shares:
        # use price last date of the dataset
        hist = tk.history(start=end_date, end=pd.to_datetime(end_date) + pd.Timedelta(days=1))
        if not hist.empty:
            mc = shares * hist["Close"].iloc[0]
    
    # Use totalAssets (AUM) if mc is None or NaN
    if mc is None or (isinstance(mc, float) and np.isnan(mc)):
        mc = info.get("totalAssets", None)
    
    rows.append({"Asset": tic, "MarketCap_or_AUM": mc})

df_marketcap = pd.DataFrame(rows)
market_caps = df_marketcap.set_index("Asset")["MarketCap_or_AUM"].to_dict()

In [ ]:
rows = []
for cls, tickers_cls in asset_classes.items():  
    # Total market cap within each class
    class_cap_total = sum(
        mc for t in tickers_cls
        if (mc := market_caps.get(t)) is not None and mc > 0
    )
    # External weight (GMP)
    class_ext_weight = GMP_percent.get(cls, 0.0)  # 0 for Factors/Volatility

    for t in tickers_cls:
        cap = market_caps.get(t, None)
        if cap and cap > 0 and class_cap_total > 0 and class_ext_weight > 0:
            internal_w = cap / class_cap_total
            prior_w = class_ext_weight * internal_w
        else:
            internal_w = 0.0
            prior_w = 0.0
        rows.append({
            "Ticker": t,
            "AssetClass": cls,
            "MarketCap_or_AUM": cap,
            "InternalWeight": internal_w,
            "ClassWeight": class_ext_weight,
            "PriorWeight": prior_w
        })

df_prior = pd.DataFrame(rows).set_index("Ticker")
print(df_prior)

In [ ]:
df_prior = df_prior.reindex(tickers)
w_mkt = df_prior["PriorWeight"].to_numpy()

# checking weights normalization
print(w_mkt.sum()) 

### Market risk-aversion coefficient
The market risk-aversion coefficient $\lambda_{\mathbf{mkt}}$ is derived from the unconstrained maximization of the mean–variance utility problem $$U(\mathbf{w})=\mathbf{w}'\boldsymbol{\mu} - \lambda \mathbf{w}'\boldsymbol{\Sigma} \mathbf{w}$$ where $\mathbf{w}$ is the vector of risky-asset weights, $\boldsymbol{\mu}$ the vector of excess expected returns, and $\boldsymbol{\Sigma}$ the return-covariance matrix. The first order condition leads to $$\boldsymbol{\mu} = 2 \lambda \boldsymbol{\Sigma} \mathbf{w}$$ 
Left-multiplying both sides by $\mathbf{w}'$ and defining $$\sigma_{\text{mkt}}^2 := \mathbf{w}' \boldsymbol{\Sigma} \mathbf{w} \;\;\; \text{and} \;\;\; \mu_{\text{mkt}} := \mathbf{w}'\boldsymbol{\mu}$$ yields the market risk aversion coefficient: $$\hat{\lambda}_{\text{mkt}}=\frac{\hat{\mu}_\text{mkt}}{\hat{\sigma}_\text{mkt}^2}$$
In practice, these quantities are estimated from weekly excess returns of $\text{SPY}$, taken as a proxy for the U.S. market. The estimators are $$\hat{\mu}_{\text{mkt}} = \frac{1}{T}\sum_{t=1}^{T}(r_t - r_{f,t}) \;\;\;\;\hat{\sigma}_{\text{mkt}}^{2}= \frac{1}{T-1}\sum_{t=1}^{T} \Bigl[(r_t - r_{f,t}) - \hat{\mu}_{\text{mkt}}\Bigr]^{2}$$ where $r_t$ denotes the weekly total return of $\text{SPY}$, $r_{f,t}$ the weekly risk-free rate, and $T$ the number of observations. The corresponding estimate of the market risk aversion is $$\lambda_{\text{mkt}}=\frac{\mu_\text{mkt}}{\sigma_\text{mkt}^2}$$

In [ ]:
mu_mkt = spy_exc.mean()
var_mkt = spy_exc.var(ddof=1)

In [ ]:
lambda_mkt = mu_mkt / (2*var_mkt)
print(lambda_mkt.round(4))
lambda_mkt = lambda_mkt.item()

### Equlibrium return
The equilibrium return is obtained by solving the reverse optimisation problem. In the classical mean–variance framework, the optimization seeks portfolio weights given an expected return vector; here, the process is inverted, and the expected return vector is inferred from a given set of market weights. In formula:
$$\boldsymbol{\pi} = 2 \lambda_{\text{mkt}}\boldsymbol{\Sigma} \mathbf{w}_{\text{mkt}}$$

In [ ]:
pi = 2 * lambda_mkt * (Sigma_lw @ w_mkt)

## Part II - Views Definition
<div style="float:left; margin:20px 0; width:500px;">
  <table style="border-collapse:collapse; font-family:Arial, sans-serif; font-size:14px;">
    <colgroup>
      <col style="width:100px; text-align:center;">
      <col style="width:90px; text-align:center;">
      <col style="width:280px; text-align:left;">
    </colgroup>
    <thead>
      <tr style="background-color:#f2f2f2;">
        <th style="border:1px solid #ccc; padding:6px; text-align:center;">Variable</th>
        <th style="border:1px solid #ccc; padding:6px; text-align:center;">Shape</th>
        <th style="border:1px solid #ccc; padding:6px; text-align:left;">Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{P}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$K \times N$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Pick Matrix</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{Q}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$K \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">View Vector</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{C}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$K \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Views Confidence Level Vector</td>
      </tr>
    </tbody>
  </table>
</div>


The pick matrix $P$ is the mathematical object that directly links the views to the corresponding assets. Each row matches a view: only the assets affected by that have a non-zero entry, and the value will be different depending on the type of view. This will become more clear after defining the views themselves and their features. The $\mathbf{Q}$ and $\mathbf{C}$ vectors respectively store the views and their confidence levels.

Here is the list of the views:
- **View 1**: *Gold will deliver a +7.0% excess return over the risk-free rate over the next year*. (Confidence level 60%)
- **View 2**: *TIPS ETF will outperform cash, reflecting a rising real yields by 2% in the next 12 months*. (Confidence level 45%)
- **View 3**: *7–10 yr Treasuries is expected to underperform High-yield credit by 5% in the next year*. (Confidence level 55%)
- **View 4**: *The US large-cap growth and real estate will outperform IG credit by 1.2% in the next year*. (Confidence level 50%)
- **View 5**: *Financial and Healthcare cyclicals are expected to outperform by 3.5% the basket of Energy, Staples & Utilities cyclicals in the next year*. (Confidence level 65%)

The pick matrix $\mathbf{P}$ is generated from an intial draft in which each view stores: (i) the tickers involved, (ii) their signs ($+1$ for the outperforming set, $–1$ for the underperforming set), and (iii) a boolean flag indicating whether the view is absolute or relative.
- Absolute views: place a $+1$ on the target asset (zeros elsewhere).
- Relative views: replace the $\pm 1$ indicators with market weights separated by sign. This approach is motivated by the fact that applying equal weights would impose disproportionately large adjustments on smaller-cap assets.

Consequently, each relative‑view row in $\mathbf{P}$ sums to zero (long basket minus short basket), while absolute‑view rows retain a single non‑zero entry. This implementation makes the views management clean, and it is easily scalable for larger portfolios or the inclusion of more views.  
 
It should be noted that the direction of the tilt induced by a view cannot be inferred from the sign of the view alone. In the absence of constraints and additional views, if the magnitude of the view is smaller than the difference between the implied equilibrium returns of two assets, the model tilts the portfolio toward the asset with the lower implied equilibrium return; if the view’s magnitude exceeds that difference, the portfolio is tilted toward the asset with the higher implied equilibrium return.
Another critical aspect is that each view is expressed on an annual basis, while the datasets have been resampled weekly. To align these frequencies, every entry in the $\mathbf{Q}$ vector is divided by 52, the number of weeks in a year.

In [ ]:
# building the views dictionary 
views = {
    'V1': {
        'legs':   {'GLD': +1},
        'relative': False
    },
    'V2': {
        'legs':   {'TIP': +1},
        'relative': False
    },
    'V3': {
        'legs':   {'HYG': +1, 'IEF': -1},
        'relative': True
    },
    'V4': {
        'legs':   {'QQQ': +1, 'VNQ': +1, 'LQD': -1},
        'relative': True
    },
    'V5': {
        'legs':   {'XLF': +1, 'XLV': +1, 'XLE': -1, 'XLP': -1, 'XLU': -1},
        'relative': True
    }
}

P_raw = pd.DataFrame(0.0, index=views.keys(), columns=tickers)
is_relative = pd.Series({name: props['relative'] for name, props in views.items()})

for name, props in views.items():
    for ticker, sign in props['legs'].items():
        P_raw.at[name, ticker] = sign

print("Skeleton P_raw:")
print(P_raw)
print("\nIs each view relative?")
print(is_relative)

In [ ]:
P = P_raw.copy()

for view_name, row in P_raw.iterrows():
    if is_relative[view_name]:
        plus_tickers  = row[row >  0].index
        minus_tickers = row[row <  0].index

        # market weights
        w_plus  = df_prior.loc[plus_tickers,  "PriorWeight"]
        w_minus = df_prior.loc[minus_tickers, "PriorWeight"]

        w_plus  = w_plus  / w_plus.sum()
        w_minus = w_minus / w_minus.sum()

        P.loc[view_name, plus_tickers]  =  w_plus.values
        P.loc[view_name, minus_tickers] = -w_minus.values

print("Final P matrix:")
print(P)
P = P.to_numpy()

In [ ]:
Q = np.array([0.070, 0.020, 0.050, 0.012, 0.035]) # views vector
C = np.array([0.60,  0.45,  0.55,  0.50,  0.65])  # confidence level vector

# adjusting frequency
Q = Q / WEEKS_PER_YEAR

N, K = len(pi), len(Q) # assets, views

## Part III - Uncertainty on Views
<div style="float:left; margin:20px 0; width:500px;">
  <table style="border-collapse:collapse; font-family:Arial, sans-serif; font-size:14px;">
    <colgroup>
      <col style="width:100px; text-align:center;">
      <col style="width:90px; text-align:center;">
      <col style="width:280px; text-align:left;">
    </colgroup>
    <thead>
      <tr style="background-color:#f2f2f2;">
        <th style="border:1px solid #ccc; padding:6px; text-align:center;">Variable</th>
        <th style="border:1px solid #ccc; padding:6px; text-align:center;">Shape</th>
        <th style="border:1px solid #ccc; padding:6px; text-align:left;">Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\boldsymbol{\mu}_{k,100\%}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">BL Return (100% certainty) Vector</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{w}_{k,100\%}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Optimal Weights (100% confidence)</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{D}_{k,100\%}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Max Departures from Market Weights</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{Tilt}_k$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Tilt caused by $k$-th view</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{w}_{k,\%}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Target weight vector</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\omega_k$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$1 \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Diagonal elements of $\boldsymbol{\Omega}$</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\boldsymbol{\Omega}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$K \times K$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Views Uncertainty Matrix</td>
      </tr>
    </tbody>
  </table>
</div>

Idzorek’s procedure quantifies view uncertainty by linking the tilt—the deviation from the equilibrium (market) portfolio implied by a fully certain view—to the investor’s stated confidence level. Formally, $$ \mathbf{Tilt}_{k}\;\approx\;\bigl(\mathbf{w}_{k,\,100\%}-\mathbf{w}_{\text{mkt}}\bigr)\,C_{k}$$ where $C_k$ is the stated confidence in view $k$, and $\mathbf{w}_{k,100\%}$ is the portfolio one would hold if that view were known with 100% certainty. Thus, a view’s weight can be interpreted as the deviation from the certainty‑view portfolio, scaled by the confidence level.

Idzorek then provides a step‑by‑step method to compute the diagonal elements $\omega_k$ of the view‑uncertainty matrix $\boldsymbol{\Omega}$, turning an otherwise abstract concept into quantities derived from tilts and confidences. Although the detailed procedures are omitted from this notebook, each step is highlighted in the code below. The complete list of procedures can be found in Appendix B of the accompanying PDF report.  
The only techincal detailed worth mentioning here is the for step 6: this step consists in a minimzation problem under the constraint $\omega_k > 0$. The definition of the problem is the following: $$\min\sum(\mathbf{w}_{k,\%} - \mathbf{w}_k)^2$$ where $\mathbf{w}_{k,\%}$ is the target weight vector based on the tilt and $$\mathbf{w}_k = \left[2\lambda_{\text{mkt}}\boldsymbol{\Sigma}\right]^{-1}\left[\boldsymbol{\Sigma}^{-1}+\frac{1}{\omega_k}\mathbf{p}_k' \mathbf{p}_k\right]^{-1}\left[\boldsymbol{\Sigma}^{-1} \boldsymbol{\pi} + \frac{Q_k}{\omega_k} \mathbf{p}_k'\right]$$ 
Here $\mathbf{p}_k$ is $k$-th row of the pick matrix. $\mathbf{p}_k' \mathbf{p}_k$ is a column-by-row product, which produces a $(N\times N)$ matrix, whose elements are $P_{i,j} = p_{k,i} \times p_{k,j}$. The constrained minimization is performed using `minimize_scalar`, which is the simplest, fastest, and most robust choice for a one-dimensional problem. This iterative process leads to the matrix $\tilde{\Omega}_{kk} = \omega_k$.  
The final step is to scale the calibrated matrix $$\boldsymbol{\Omega} = \tau \, \tilde{\boldsymbol{\Omega}}$$
In this framework, $\tau$ appears only as a constant factor that cancels algebraically in the posterior mean. Consequently, the combined expected return $\boldsymbol{\mu}_{\text{BL}}$ is invariant to the specific value of $\tau$, making its choice operationally irrelevant.

Note: Throughout the implementation, linear systems are solved using `np.linalg.solve` rather than by explicitly computing matrix inverses. The `solve` routine internally applies an LU decomposition of the coefficient matrix, which is both numerically more stable and computationally more efficient than forming $\boldsymbol{\Sigma}^{-1}$ explicitly. The only exception arises when the full inverse of $\boldsymbol{\Sigma}$ is required as a matrix (e.g., in the computation of implied weights), in which case `np.linalg.inv` is used.

In [ ]:
tau = 1  # results are independent from tau 
S_inv = np.linalg.inv(Sigma_lw)

def implied_weights(p_k, Q_k, omega, lam):
    lhs = S_inv + (1.0/omega) * np.outer(p_k, p_k)
    rhs = S_inv @ pi + (Q_k/omega) * p_k
    x   = np.linalg.solve(lhs, rhs)

    return np.linalg.solve(2 * lam * Sigma_lw, x)

In [ ]:
def uncertainty_matrix():
    """Idzorek procedure for Ω definition"""
    mu_100    = np.zeros((K, N))
    w_100     = np.zeros((K, N))
    D_100     = np.zeros((K, N))
    tilt      = np.zeros((K, N))
    w_tilt    = np.zeros((K, N))
    Omega     = np.zeros((K, K))
    omega_opt = np.empty(K)
    
    for k in range(K):
        p_k = P[k]  
        Q_k = Q[k]  

        num = Sigma_lw @ p_k           
        den = p_k @ Sigma_lw @ p_k     
        delta = Q_k - (p_k @ pi)

        mu_100[k] = pi + (num / den) * delta # Step 1
        w_100[k] = np.linalg.solve(2 * lambda_mkt * Sigma_lw, mu_100[k]) # Step 2
        D_100[k] = w_100[k] - w_mkt # Step 3
        tilt[k] = D_100[k] * C[k]   # Step 4
        w_tilt[k] = w_mkt + tilt[k] # Step 5

        def _objective(omega):
            w_k = implied_weights(p_k, Q_k, omega, lambda_mkt)
            return np.sum((w_k - w_tilt[k])**2)

        res = minimize_scalar(_objective, bounds=(1e-8, 1e8), method='bounded')   
        omega_opt[k] = res.x  # Step 6

        Omega[k, k] = omega_opt[k]   
    
    return Omega

## Part IV - BL Posterior and Weights 
<div style="float:left; margin:20px 0; width:500px;">
  <table style="border-collapse:collapse; font-family:Arial, sans-serif; font-size:14px;">
    <colgroup>
      <col style="width:100px; text-align:center;">
      <col style="width:90px; text-align:center;">
      <col style="width:280px; text-align:left;">
    </colgroup>
    <thead>
      <tr style="background-color:#f2f2f2;">
        <th style="border:1px solid #ccc; padding:6px; text-align:center;">Variable</th>
        <th style="border:1px solid #ccc; padding:6px; text-align:center;">Shape</th>
        <th style="border:1px solid #ccc; padding:6px; text-align:left;">Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\boldsymbol{\mu}_{\text{BL}}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">BL Return Vector</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\lambda$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$1 \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">Investor's Risk Aversion</td>
      </tr>
      <tr>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$\mathbf{w}_{\text{BL}}$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:center;">$N \times 1$</td>
        <td style="border:1px solid #ccc; padding:6px; text-align:left;">BL Optimal Weights</td>
      </tr>
    </tbody>
  </table>
</div>

### Posterior returns 
For the posterior return vector, the stable representation proposed by Meucci (2010) is adopted: $$\boldsymbol{\mu}_{\text{BL}} = \boldsymbol{\pi} + \tau \boldsymbol{\Sigma} \mathbf{P}' \bigl(\tau \mathbf{P} \boldsymbol{\Sigma} \mathbf{P}'+\boldsymbol{\Omega}\bigr)^{-1} \bigl(\mathbf{Q}-\mathbf{P}\boldsymbol{\pi}\bigr)$$ Since $\boldsymbol{\Omega}$ was defined to be proportional to $\tau$, it can be factored out and canceled from the posterior return.  

In [ ]:
def posterior_ret(Omega):
    """
    Black-Litterman posterior return using stable representation.
    """
    S_tau = tau * Sigma_lw 

    Kmat = P @ S_tau @ P.T + Omega * tau
    rhs  = Q - P @ pi
    adj  = np.linalg.solve(Kmat, rhs)          
    mu_BL = pi + S_tau @ P.T @ adj             

    return mu_BL

In [ ]:
Omega = uncertainty_matrix()  
mu_BL = posterior_ret(Omega)

To illustrate the effect of the Black–Litterman update, the equilibrium returns $\boldsymbol{\pi}$, the posterior returns $\boldsymbol{\mu}_{\text{BL}}$, and their differences $\boldsymbol{\mu}_{\text{BL}} - \boldsymbol{\pi}$ are reported side‐by‐side (annualized). In addition, for each investor profile, the tilt vector $$ \mathbf{Tilt} = \mathbf{w}_{\text{BL}} - \mathbf{w}_{\text{mkt}} $$ is shown. These tilts highlight how each portfolio deviates from the market‐capitalization benchmark, providing a direct view of how the incorporation of views shifts allocations.

In [ ]:
returns = pd.DataFrame({
    "mu_BL [%]"      : (mu_BL * WEEKS_PER_YEAR)* 100,                 
    "pi [%]"         : (pi * WEEKS_PER_YEAR) *100,
    "mu_BL - pi [%]" : ((mu_BL - pi) * WEEKS_PER_YEAR) * 100
}, index=tickers)

print(returns)

$\lambda$ is the investor's risk-aversion coefficient. Three different investor categories are considered, Kelly ($\lambda = 0.005$), Market ($\lambda=1.12$) and Trustee ($\lambda=3$). The values adopted are those suggested in the final project workshop. 

In [ ]:
investor_profile = ["Kelly", "Average", "Trustee"]
lambda_df = pd.DataFrame(
    {"lambda": [0.005, 1.12, 3.0]},
    index=investor_profile
)
    
investors = lambda_df.index.to_list()     
lam_arr   = lambda_df["lambda"].to_numpy()
I = len(investors) # number of investor types

The optimal weights are then computed using the unconstrained mean-variance approach, which has the following analytical solution:
$$\mathbf{w}^* = \frac{1}{2\lambda}\boldsymbol{\Sigma}^{-1} \boldsymbol{\mu}_{\text{BL}}$$
This example illustrates the simplest form of portfolio optimization and is presented as a concise showcase. Later, in a dedicated section, this approach will be revisited by applying constraints, and comparing the results with other two techniques. The three weight sets obtained for the different investor profiles are stored together with equal–weight (`w_eq`) and market–weight (`w_mkt`) portfolios for comparison. Both raw and normalized weights are retained. Since equal and market weights are already positive and sum to one, the normalized form facilitates direct comparison, even though short positions may still occur in the unconstrained solutions before normalization. The Black-Litterman weights are distinguished by a suffix identifying the investor profile: `k` for Kelly, `a` for Average and `t` for Trustee.

In [ ]:
wBL_all = np.empty((I, N))
for i, lam in enumerate(lam_arr):
    wBL_all[i] = np.linalg.solve(2 * lam * Sigma_lw, mu_BL)

In [ ]:
weights_df = pd.DataFrame(index=tickers)
weights_df['w_eq']  = np.full(N, 1/N)       
weights_df['w_mkt'] = w_mkt                 

suffix_list = {'Kelly': 'k', 'Average': 'a', 'Trustee': 't'}

for row_idx, profile in enumerate(lambda_df.index):
    suffix = suffix_list[profile]
    weights_df[f'w_BL_{suffix}'] = wBL_all[row_idx]
    weights_df[f'w_BL_{suffix}_norm'] = wBL_all[row_idx] / wBL_all[row_idx].sum()
    weights_df[f'w_BL_{suffix}_tilt'] = weights_df[f'w_BL_{suffix}'] - weights_df['w_mkt']

print(weights_df.round(4))

In [ ]:
print(f"Kelly: sum of weights {weights_df['w_BL_k'].sum().round(4)*100}%")
print(f"Average: sum of weights {weights_df['w_BL_a'].sum().round(4)*100}%")
print(f"Trustee: sum of weights {weights_df['w_BL_t'].sum().round(4)*100}%")

Plotting all the normalzied weights collected for visual comparison

In [ ]:
cols_to_plot = ['w_eq','w_mkt','w_BL_k_norm','w_BL_a_norm','w_BL_t_norm']
weights_norm = weights_df[cols_to_plot]

labels  = ['Equal Weight', 'Market Cap',
           'BL (Kelly)', 'BL (Average)', 'BL (Trustee)']
colors  = ['lightgray', 'orange', '#6baed6', '#31a354', '#756bb1']

x = np.arange(N)
bar_width = 0.8 / len(cols_to_plot)

fig, ax = plt.subplots(figsize=(12, 6))

for i, (col, lab, colr) in enumerate(zip(cols_to_plot, labels, colors)):
    ax.bar(
        x + (i - (len(cols_to_plot)-1)/2) * bar_width,
        weights_norm[col].values,
        width  = bar_width,
        label  = lab,
        color  = colr,
    )

ax.set_xticks(x)
ax.set_xticklabels(weights_norm.index)
for xi in x[:-1]:
    ax.axvline(x=xi + 0.5, linestyle='--', linewidth=0.5,
               color='gray', alpha=0.7)

ax.set_xlim(-0.5, N - 0.5)
ax.set_ylabel('Normalized weight')
ax.set_title('Portfolio Allocations (Normalized)')
ax.legend(ncol=3)
plt.tight_layout()
plt.show()

---

# Optimization

Three optimization methods are examined:  
1. **Mean–Variance (Markowitz)** 
2. **Maximum Sharpe Ratio**  
3. **Risk Budgeting**

For consistency and to address robustness concerns, each method is subject to the following constraints:  
- **Budget constraint**:  $$\sum_{i=1}^{N} \text{w}_i = 1$$
- **Box constraint** (no shorting or leverage): $$0 \le \text{w}_i \le 1 \quad \forall\,i = 1,2,\dots,N$$

The $\text{w}_i \le 1$ condition is mathematically redundant if the budget and non-negativity constraints are satisfied, but it is still reported as a diagnostic to catch potential numerical violations.

Analysis is restricted to the "Average" investor profile, with the risk-aversion parameter fixed at $\lambda = 1.12$. 

In [ ]:
lambda_avg = lambda_df.loc["Average", "lambda"]

opt_weights = pd.DataFrame()
opt_weights['w_eq'] = weights_df['w_eq']
opt_weights['w_mkt'] = weights_df['w_mkt']

### Performance assessment

Algorithm performance is evaluated on three features: **feasibility**, **numerical quality**, and **robustness**.

- **Feasibility** — Solutions are checked against the budget and box constraints with tight tolerances. The following violations are reported, which should be numerically close to zero for a valid optimum: $$\underbrace{|1 - \mathbf{1}' \mathbf{w}|}_{\text{budget}},\;\;\;\;\underbrace{\max(0,-\min_i \text{w}_i)}_{\text{no shorts}}, \;\;\;\; \underbrace{\max(0,\max_i \text{w}_i-1)}_{\text{no over-weights}}$$ In practice, violations $\le 10^{-8}$ are considered "tight". 

- **Numerical quality** - Method-specific metrics are reported to quantify how well the computed solution optimizes its stated objective: 
    - *mean-variance*: primal objective, dual objective, and primal–dual gap.
    - *max sharpe*: Dinkelbach residual, final inner solver first-order optimality and maximum constraint violation.
    - *risk budgeting*: final value of the risk-budgeting objective.

- **Robustness** — Stability is assessed by re-solving after small, controlled perturbations of $(\boldsymbol{\mu},\boldsymbol{\Sigma})$. For each method, $R$ independent perturbations are generated and solved; here $R=5$, meaning that each optimisation is re-run on five distinct jittered inputs. The distribution of resulting weights, objective values, and solve times is summarised. Robust algorithms exhibit small dispersion: solutions and times should not vary erratically under mild input noise. Perturbations are generated as follows: 
    1. Symmetrization and Positive Semi-Definiteness (PSD) enforcement - The covariance estimate is first symmetrized, then shifted on the diagonal if its smallest eigenvalue is below a tolerance $\varepsilon$. Concretely, if $\lambda_{\min}(\boldsymbol{\Sigma})<\varepsilon$ then set $\boldsymbol{\Sigma} \leftarrow \boldsymbol{\Sigma} + (\varepsilon-\lambda_{\min})\mathbb{I}$, which ensures $\lambda_{\min}(\boldsymbol{\Sigma})\ge \varepsilon$ (hence $\boldsymbol{\Sigma}$ is PSD).
    2. Mean jitter - perturb expected returns with i.i.d. Gaussian noise scaled by the cross‑sectional dispersion of $\boldsymbol{\mu}$: $$\boldsymbol{\mu}_j = \boldsymbol{\mu} + \mu_{scale} \cdot \max(10^{-12}, \text{std}_{\mu} )\epsilon_{\mu},\;\;\;\;\epsilon_{\mu}\sim\mathcal{N}(0,1)$$ By default, $\mu_{\text{scale}}=1\%$.
    3. Covariance jitter: draw a symmetric noise matrix $\mathbf{E}=(\mathbf{Z}+\mathbf{Z}')/2$ with $Z_{ij}\sim\mathcal{N}(0,1)$ and scale it by the average covariance level using the Frobenius norm: $$\text{scale} = \sigma_{scale}\frac{||\boldsymbol{\Sigma}||_2}{N},\;\;\;\; \text{Proj}_{\text{PSD}}=(\boldsymbol{\Sigma} + \text{scale} \cdot \mathbf{E})$$ where by default $\sigma_{\text{scale}}=1%$ and $\operatorname{Proj}_{\text{PSD}}$ denotes the eigenvalue‑flooring step in (1).

In [ ]:
# Pertubation functions
def _ensure_psd(S, eps=1e-10):
    S = (S + S.T)/2
    w, V = np.linalg.eigh(S)
    m = w.min()
    if m < eps:
        S = S + (eps - m) * np.eye(S.shape[0])
    return (S + S.T)/2

def _jitter_inputs(mu, Sigma, mu_scale=0.01, sig_scale=0.01, rng=None):
    rng = np.random.default_rng() if rng is None else rng
    # jitter mu 
    mu_j = mu + mu_scale * max(1e-12, float(np.std(mu))) * rng.standard_normal(mu.shape)
    
    # jitter Sigma 
    E = rng.standard_normal(Sigma.shape); E = (E + E.T)/2
    scale = sig_scale * (np.linalg.norm(Sigma, 'fro') / Sigma.shape[0]) 
    Sigma_j = _ensure_psd(Sigma + scale * E)
    
    return mu_j, Sigma_j

For each of the $R$ jittered runs, the robustness routine records:
- `iters` — solver iteration count.
- `gap` — primal–dual optimality gap at termination.
- `feas` — budget constraint violation $|1 - \mathbf{1}' \mathbf{w}|$
- `lb` — lower-bound violation $\max(0,-\min_i \text{w}_i)$
- `ub` — upper-bound violation $\max(0,\max_i \text{w}_i-1)$ (redundant but included for diagnostics).
- `wL1` — $\ell^1$ distance from baseline weights: total absolute reallocation caused by jitter.
- `wLinf` — $\ell^{\infty}$ distance from baseline weights: largest single‐asset change.

Summary statistics are then computed: medians for runtimes, iteration counts, gaps, and weight deviations; maxima for feasibility violations, to report the worst‐case drift observed across the $R$ perturbations.

## Markowitz Mean–Variance

With the constraints defined, the portfolio choice problem becomes a Quadratic Program (QP): a quadratic objective function subject to linear constraints.  
The utility function is  
$$
U(\mathbf{w}) = \mathbf{w}'\boldsymbol{\mu}_{\text{BL}} - \lambda \mathbf{w}' \boldsymbol{\Sigma}\mathbf{w}
$$

The `CVXOPT` package is used to solve this via its `cvxopt.solvers.qp` function, which addresses problems of the form

$$
\begin{aligned}
&\underset{\mathbf{x}}{\mathrm{minimize}}
&& \tfrac12\,\mathbf{x}'\mathbf{H}\,\mathbf{x} \;+\; \mathbf{f}' \\[6pt]
&\text{subject to}
&& \mathbf{G}\,\mathbf{x} \;\le\; \mathbf{h},\\
&&& \mathbf{A}\,\mathbf{x} = b.
\end{aligned}
$$

All inputs are provided as `cvxopt.matrix` objects. Note that even though they are named "matrix" objects they can also be vectors or scalars. To match the Markowitz objective, $\mathbf{H} = 2\,\lambda\,\boldsymbol{\Sigma}$ while $\mathbf{f} = -\boldsymbol{\mu}_{\text{BL}}$.  
The budget constraint implementation is straghtforward ($\mathbf{A}=[1,1,\dots,1],\;b=1$ ), while the box constraints $0 \le x_i \le 1$ require a little more structure:

- $\mathbf{G}$ is a $(2N \times N)$ matrix formed by stacking the $(N\times N)$ identity matrix above its negation:
  $$
    \mathbf{G} = \begin{pmatrix}
      \mathbb{I}_N \\[4pt]
      -\mathbb{I}_N
    \end{pmatrix}
  $$
- $\mathbf{h}$ is a $2N$-vector with the first $N$ entries equal to 1 and the last $N$ entries equal to 0:
  $$
    \mathbf{h} = \begin{pmatrix}
      \mathbf{1}_N \\[4pt]
      \mathbf{0}_N
    \end{pmatrix}
  $$

Matrix multiplication then leads to  
$$
\mathbb{I}_N\,x \;\le\; \mathbf{1}_N \quad\Longrightarrow\quad x_i \le 1,\;\;\forall \,i \\
-\,\mathbb{I}_N\,x \;\le\; \mathbf{0}_N \quad\Longrightarrow\quad -x_i \le 0,\;\;\forall \,i,
$$  
which together impose $0 \le x_i \le 1$.

Numerical quality is assessed from the solver’s status flag, the number of iterations to convergence, and the duality gap. The primal objective corresponds to the QP itself, while the dual objective is obtained from the Lagrangian dual problem. `CVXOPT` reports both values: for any feasible problem, $$\text{primal obj} \geq \text{dual obj}$$ and at optimality the duality gap $$\text{gap} = \text{primal obj} - \text{dual obj} \approx 0$$ A small gap certifies that the computed solution is close to optimal. 

In [ ]:
def mean_variance_qp(mu, Sigma, lam):
    """
    Solve a long-only, fully-invested mean-variance optimization via CVXOPT QP.

    Inputs:
        mu    : excess return vector
        Sigma : covariance matrix of excess returns
        lam   : risk aversion parameter

    Output:
        dict with solver status, and optimal weights
    """

    H = 2 * lam * Sigma
    f = -mu

    H_cvx = matrix(H)
    f_cvx = matrix(f)

    # Bounds 
    G_cvx = matrix(np.vstack((np.eye(N), -np.eye(N))))
    h_cvx = matrix(np.hstack((np.ones(N), np.zeros(N))))

    # Budget constraint
    A_cvx = matrix(np.ones((1, N)))
    b_cvx = matrix(1.0)

    solvers.options.update({
        'show_progress': False,
        'abstol': 1e-9,
        'reltol': 1e-9,
        'feastol': 1e-9,
        'maxiters': 100
    })

    # Solving QP
    sol = solvers.qp(H_cvx, f_cvx, G_cvx, h_cvx, A_cvx, b_cvx)

    # Numerical quality
    w = np.array(sol['x']).ravel()
    status = sol['status']
    iters = sol['iterations']
    primal = sol['primal objective']
    dual   = sol['dual objective']
    gap    = primal - dual  

    # Feasibility checks
    feas_budget = abs(w.sum() - 1.0)
    feas_lower  = max(0.0, -w.min())
    feas_upper  = max(0.0,  w.max() - 1.0)

    # Outputs
    report = {
        'status': status,
        'iters': iters,
        'primal_obj': primal,
        'dual_obj': dual,
        'gap': gap,
        'feas_budget': feas_budget,
        'feas_lb_viol': feas_lower,
        'feas_ub_viol': feas_upper,
        'weights': w
    }
    return report

rep = mean_variance_qp(mu_BL, Sigma_lw, lambda_avg)

print(f"Status: {rep['status']}, iters: {rep['iters']}")
print(f"Feasibility  | budget: {rep['feas_budget']:.2e}, "
      f"lb: {rep['feas_lb_viol']:.2e}, ub: {rep['feas_ub_viol']:.2e}")
print(f"Objectives   | primal: {rep['primal_obj']:.8f}, dual: {rep['dual_obj']:.8f}, gap: {rep['gap']:.2e}")

opt_weights['w_mv'] = rep['weights']

In [ ]:
# Robustness 
def mv_input_sensitivity(mu, Sigma, lam, R=5, mu_scale=0.01, sig_scale=0.01, seed=0):
    """
    Assess mean-variance solution sensitivity to small perturbations in μ and Σ.

    Inputs:
        mu, Sigma : base excess returns and covariance
        lam       : risk aversion parameter
        R         : number of jittered runs

    Output:
        tuple of (all run results DataFrame, aggregated summary, base run report)
    """
    
    base = mean_variance_qp(mu, Sigma, lam)
    w0 = base['weights']
    rows = []
    rng = np.random.default_rng(seed)

    # jitters
    for r in range(R):
        mu_j, Sig_j = _jitter_inputs(mu, Sigma, mu_scale, sig_scale, rng)
        rep = mean_variance_qp(mu_j, Sig_j, lam)
        rows.append(dict(
            run=r,
            iters=rep['iters'],
            gap=rep['gap'],
            feas=rep['feas_budget'],
            lb=rep['feas_lb_viol'],
            ub=rep['feas_ub_viol'],
            wL1=float(np.sum(np.abs(rep['weights'] - w0))),
            wLinf=float(np.max(np.abs(rep['weights'] - w0)))
        ))
    df = pd.DataFrame(rows)
    summary = df.agg({
        'iters':'median', 'gap':'median',
        'feas':'max', 'lb':'max', 'ub':'max',
        'wL1':'median', 'wLinf':'median'
    }).rename({'feas':'feas_max', 'lb':'lb_max', 'ub':'ub_max'})
    print("Mean-Variance: input sensitivity (medians / max violations)")
    print(summary.round(6).to_string())
    return df, summary, base

df_rob, sum_rob, mv_base = mv_input_sensitivity(mu_BL, Sigma_lw, lambda_avg, R=5, mu_scale=0.01, sig_scale=0.01)

## Maximum Sharpe Ratio

This optimization strategy aims to maximize the portfolio's Sharpe ratio. The utility function is $$\text{SR}(\mathbf{w}) = \frac{\mathbf{w}'\boldsymbol{\mu}_{\text{BL}}}{\sqrt{\mathbf{w}'\boldsymbol{\Sigma}\mathbf{w}}}$$
Due to the square root at the denominator this problem is not convex, so it isn't possible to use the same QP technique as before. However, it belongs to a particular class called *fractional programs*, which can be addressed numerically with iterative methods. Here, Dinkelbach’s algorithm is applied. This method transforms the fractional objective into a sequence of convex subproblems:
$$
\begin{aligned}
&\underset{\mathbf{w}}{\mathrm{maximize}}
&& \,\mathbf{w}'\boldsymbol{\mu}_{\text{BL}} - y_{(k)} \sqrt{\mathbf{w}'\boldsymbol{\Sigma} \mathbf{w}}\\[6pt]
&\text{subject to}
&& \sum_i\text{w}_i = 1 \\
&&& \text{w}_i > 0\;\;\forall \,i = 1,2,\dots,N.
\end{aligned}
$$
where the parameter $$y_{(k)} = \frac{\mathbf{w}_{(k)}'\boldsymbol{\mu}_{\text{BL}}}{\sqrt{\mathbf{w}_{(k)}' \boldsymbol{\Sigma}\mathbf{w}_{(k)}}}$$ will be updated at each iteration. In this equation the subscript $(k)$ is the iteration index, not the element of a vector.

The algorithm proceeds as follows:
- **Step 0**: Initialize $\mathbf{w}_{(0)}$ (e.g., equal weights) and compute $y_{(0)}$.
- **Step 1**: Solve the convex problem defined above to find the new set of weights. These will be passed to the next iteration to improve convergence speed (warm start).
The convex problem is solved by changing the sign of the objective function and using `scipy.minimize` with the method `trust-constr`, which offers a good balance of robustness and speed. This method requires the budget constraint to be passed as a `LinearConstraint` object with three arguments: the row vector $\mathbf{A}$ of shape $(1 \times N)^{\dagger}$, the lower bound (`lb`) and the upper bound (`ub`), which is then interpeted as $\text{lb} \le \mathbf{A}\,\mathbf{x}\le \text{ub}$ (where $\mathbf{x}$ is the independent variables vector). Equality constraints can be obtained using equal bounds. The solver also accepts the objective Jacobian, which materially reduces runtime.
- **Step 2**: Update $y_{(k+1)}$ 
- **Step 3**: Convergence check: if $|y_{(k+1)} - y_{(k)}|<\varepsilon$ terminate; otherwise repeat from step 1 until convergence.

In practice, the procedure converges in a small number of outer iterations.

$^\dagger$ The class actually accepts a matrix of shape $(m \times n)$, where $m$ is the number of constraints and $n$ the number of independent variables.

Numerical quality is evaluated from these diagnostics: 
- Dinkelbach residual: $r = |y_{(k+1)}-y_{(k)}|$
- Final inner solver optimality — first-order optimality measure (`optimality`) from the last `trust-constr` solve.
- Final inner solver constraint violation — maximum absolute constraint violation (`constr_violation`) from the last inner solve.

Performance metrics are reported separately: outer iteration count and last/total inner iteration counts.

In [ ]:
# inner problem 
def solve_convex(mu, Sigma, y, w0, bounds, cons):
    """
    Solve the convex subproblem in the Dinkelbach Max Sharpe method.
    
    Inputs:
        mu, Sigma : excess returns and covariance
        y         : current Sharpe ratio estimate
        w0        : starting weights
        bounds    : box constraints
        cons      : budget constraints

    Output:
        SciPy OptimizeResult of the inner solve
    """

    def _obj(w):
        return -(w @ mu - y * np.sqrt(w @ Sigma @ w))
    def _jac(w):
        v = w @ Sigma @ w
        s = np.sqrt(max(v, 1e-18)) # avoid zero division
        return -(mu - y * (Sigma @ w) / s)
    res = minimize(_obj, w0, 
                   method='trust-constr', 
                   jac=_jac,
                   bounds=bounds, 
                   constraints=cons,
                   options={'maxiter': 500})
    if not res.success:
        raise RuntimeError("Inner solve failed: " + res.message)
    return res

def find_y(mu, Sigma, w):
    return (w @ mu) / np.sqrt(max(w @ Sigma @ w, 1e-18)) # avoid zero division

# Dinkelbach algorithm (outer problem)
def max_sharpe_report(mu, Sigma, w0, bounds, cons, tol=1e-6, max_outer=30):
    """
    Compute the maximum Sharpe ratio portfolio using the Dinkelbach algorithm.
    
    Inputs:
        mu, Sigma : excess returns and covariance
        w0        : initial weights
        bounds    : box constraints
        cons      : budget constraints

    Output:
        dict with iteration counts, feasibility checks, and optimal weights
    """
    
    y = find_y(mu, Sigma, w0)
    w = w0.copy()
    inner_nits = []
    last_inner = None
    dres = np.nan

    for k in range(max_outer):
        res = solve_convex(mu, Sigma, y, w, bounds, cons)
        w_new = res.x
        inner_nits.append(res.nit)
        last_inner = res

        y_new = find_y(mu, Sigma, w_new)
        dres = abs(y_new - y)
        w, y = w_new, y_new
        if dres < tol:
            break

    # feasibility
    feas_budget = abs(1.0 - w.sum())
    feas_lower  = max(0.0, -w.min())
    feas_upper  = max(0.0,  w.max() - 1.0)

    # numerical quality (outer/inner + residuals)
    outer_iters = k + 1
    inner_total = int(np.sum(inner_nits))
    inner_last  = int(inner_nits[-1]) if inner_nits else 0
    inner_opt   = getattr(last_inner, 'optimality', np.nan)
    inner_cviol = getattr(last_inner, 'constr_violation', np.nan)

    # Outputs
    report = {
        'weights': w,
        'outer_iters': outer_iters,
        'inner_iters_last': inner_last,
        'inner_iters_total': inner_total,
        'dinkelbach_residual': dres,          
        'feas_budget': feas_budget,
        'feas_lb_viol': feas_lower,
        'feas_ub_viol': feas_upper,
        'inner_optimality': inner_opt,       
        'inner_constr_violation': inner_cviol,
    }
    
    return report

bounds = [(0.0, 1.0)] * N
cons   = LinearConstraint(np.ones((1, N)), 1.0, 1.0)
w0     = np.full(N, 1.0/N)

rep = max_sharpe_report(mu_BL, Sigma_lw, w0, bounds, cons)

print(f"Outer iters: {rep['outer_iters']}, inner (last/total): {rep['inner_iters_last']} / {rep['inner_iters_total']}")
print(f"Feasibility  | budget: {rep['feas_budget']:.2e}, lb viol: {rep['feas_lb_viol']:.2e}, ub viol: {rep['feas_ub_viol']:.2e}")
print(f"Numerical    | Dinkelbach residual: {rep['dinkelbach_residual']:.2e}, "
      f"inner optimality: {rep['inner_optimality']:.2e}, inner constr viol: {rep['inner_constr_violation']:.2e}")

opt_weights['w_ms'] = rep['weights']

In [ ]:
# Robustness
def ms_input_sensitivity(mu, Sigma, w0, bounds, cons, R=5, mu_scale=0.01, sig_scale=0.01, seed=0):
    """
    Test sensitivity of the Max Sharpe solution to small perturbations in μ and Σ.

    Inputs:
        mu, Sigma : base excess returns and covariance
        w0        : initial weights
        bounds    : box constraints
        cons      : budget constraints
        R         : number of jittered runs

    Output:
        tuple (runs DataFrame, aggregated summary, base run report)
    """
    
    base = max_sharpe_report(mu, Sigma, w0, bounds, cons)
    w_base = base['weights']
    rows = []
    rng = np.random.default_rng(seed)

    for r in range(R):
        mu_j, Sig_j = _jitter_inputs(mu, Sigma, mu_scale, sig_scale, rng)
        rep = max_sharpe_report(mu_j, Sig_j, w0, bounds, cons)
        rows.append(dict(
            run=r,
            outer=rep['outer_iters'],
            inner=rep['inner_iters_total'],
            dres=rep['dinkelbach_residual'],
            feas=rep['feas_budget'],
            lb=rep['feas_lb_viol'],
            ub=rep['feas_ub_viol'],
            wL1=float(np.sum(np.abs(rep['weights'] - w_base))),
            wLinf=float(np.max(np.abs(rep['weights'] - w_base)))
        ))

    df = pd.DataFrame(rows)
    summary = df.agg({
        'outer':'median', 'inner':'median', 'dres':'median',
        'feas':'max', 'lb':'max', 'ub':'max', 'wL1':'median', 'wLinf':'median'
    }).rename({'feas':'feas_max','lb':'lb_max','ub':'ub_max'})

    print("Max-Sharpe: μ, Σ input sensitivity (medians / max violations)")
    print(summary.round(6).to_string())
    return df, summary, base

df_ms, sum_ms, base_ms = ms_input_sensitivity(
    mu_BL, Sigma_lw, w0, bounds, cons,
    R=5, mu_scale=0.01, sig_scale=0.01, seed=42
)

## Risk budgeting

In this approach, portfolio weights are chosen so that each asset’s contribution to portfolio Expected Shortfall (ES) matches a target “budget” vector $\boldsymbol{\beta}$.  
First, compute the ES tail factor $$\varphi_{\text{ES}} = \frac{\phi (z_c)}{c}, \;\;\;\; z_c = \Phi^{-1}(1-c)$$ 
for confidence level $c$, where $\Phi$ is the standard normal CDF. In line with the Fundamental Review of the Trading Book (FRTB) directives, $c=0.025$ is used, corresponding to a 97.5% confidence level.

Given weights $\mathbf{w}$, the ES risk contribution of asset $i$ is
$$\text{RC}_i (\mathbf{w}) = \text{w}_i \left( -\mu_{\text{BL},i} + \varphi_{\text{ES}} \cdot \frac{\left(\boldsymbol{\Sigma} \mathbf{w}\right)_i}{\sqrt{\mathbf{w}'\boldsymbol{\Sigma}\mathbf{w}}}\right)$$
and the corresponding share is
$$S_i = \frac{\text{RC}_i}{\sum_j \text{RC}_j},$$
so that $\sum_i S_i = 1$.

Weights are obtained by minimizing the squared deviation between the normalized contributions and the target budgets, under the usual constraints
$$
\begin{align*}
\min_{\text{w}} & \sum_i \bigl(S_i - \beta_i\bigr)^2\\[6pt]
\text{subject to} \;\;\;& \sum_i\text{w}_i = 1 \\
 &\text{w}_i > 0\;\;\forall \,i = 1,2,\dots,N.
\end{align*}
$$

This yields a risk-budgeted allocation whose ES contribution shares match $\boldsymbol{\beta}$.
The problem is nonlinear and non-convex, so a Sequential Quadratic Programming (SQP) scheme is adopted. In SciPy, `SLSQP` implements SQP by, at each iteration, building a quadratic model of the Lagrangian and a first-order linearization of the constraints; the resulting quadratic program provides a search direction that is accepted via a line-search/merit function.

Numerical quality is evaluated based on the solver’s reported status (`success`), the achieved objective value (smaller is better, ideally near zero), and feasibility checks for the final weights (budget and box violations, as in the general section). Solver tolerances are those passed to SLSQP.

In [ ]:
# ES tail factor
def es_factor(c=0.025):  # 97.5% tail
    zc = norm.ppf(1 - c)
    return norm.pdf(zc) / c

# Risk contribution vector
def rc_vector_es(w, mu, Sigma, factor):
    port_sd     = np.sqrt(w @ Sigma @ w)
    marginal_sd = (Sigma @ w) / port_sd
    return w * (-mu + factor * marginal_sd)

def es_risk_budgeting_contributions(mu, Sigma, c=0.025, budget=None, tol=1e-12, maxiter=1000):
    """
    Compute an Expected Shortfall (ES) risk-budgeting portfolio via SLSQP.

    Inputs:
        mu, Sigma : excess returns and covariance
        c         : ES confidence level (e.g., 0.025 for 97.5% ES)
        budget    : target risk budget vector (defaults to equal budget)

    Output:
        dict with iteration count, feasibility checks, and optimal weights
    """
    
    mu    = np.asarray(mu, dtype=float)
    Sigma = np.asarray(Sigma, dtype=float)
    N     = mu.size

    # budget weights (market)
    budget = np.asarray(budget, dtype=float)

    factor = es_factor(c)

    def _objective(w):
        rc    = rc_vector_es(w, mu, Sigma, factor)
        total = rc.sum()
        return np.sum((rc/total - budget)**2)

    # constraints
    cons   = ({'type':'eq', 'fun': lambda w: np.sum(w) - 1.0},)  
    bounds = [(0.0, 1.0)] * N                                    
    w0     = np.full(N, 1.0/N)

    res = minimize(_objective, w0, method="SLSQP",
                   bounds=bounds, constraints=cons,
                   options={'ftol': tol, 'maxiter': maxiter})

    if not res.success:
        raise RuntimeError("ES risk-budgeting failed: " + res.message)

    w = res.x

    # feasibility checks
    feas_budget = abs(1.0 - w.sum())
    feas_lower  = max(0.0, -w.min())
    feas_upper  = max(0.0,  w.max() - 1.0)

    # Outputs
    report = {
        'weights': w,
        'nit': res.nit,
        'feas_budget': feas_budget,
        'feas_lb_viol': feas_lower,
        'feas_ub_viol': feas_upper,
        'objective': _objective(w)
    }

    return report

rep = es_risk_budgeting_contributions(
    mu=mu_BL, Sigma=Sigma_lw, c=0.025, budget=w_mkt
)

print(f"SLSQP iterations: {rep['nit']}")
print(f"Feasibility | budget: {rep['feas_budget']:.2e}, "
      f"lb viol: {rep['feas_lb_viol']:.2e}, ub viol: {rep['feas_ub_viol']:.2e}")

opt_weights['w_rb'] = rep['weights']

In [ ]:
# Robustness
def rb_input_sensitivity(mu, Sigma, budget, R=5, mu_scale=0.01, sig_scale=0.01, seed=0, c=0.025):
    """
    Test sensitivity of the ES risk-budgeting solution to small perturbations in μ and Σ.

    Inputs:
        mu, Sigma : base excess returns and covariance
        budget    : target risk budget vector
        R         : number of jittered runs
        c         : ES confidence level

    Output:
        tuple (runs DataFrame, aggregated summary, base run report)
    """
    
    base = es_risk_budgeting_contributions(mu, Sigma, c=c, budget=budget)
    w_base = base['weights']

    rows = []
    rng = np.random.default_rng(seed)
    for r in range(R):
        mu_j, Sig_j = _jitter_inputs(mu, Sigma, mu_scale, sig_scale, rng)  # uses your existing helper
        rep = es_risk_budgeting_contributions(mu_j, Sig_j, c=c, budget=budget)
        rows.append(dict(
            run=r,
            nit=rep['nit'],
            feas=rep['feas_budget'],
            lb=rep['feas_lb_viol'],
            ub=rep['feas_ub_viol'],
            obj=rep['objective'],
            wL1=float(np.sum(np.abs(rep['weights'] - w_base))),
            wLinf=float(np.max(np.abs(rep['weights'] - w_base)))
        ))

    df = pd.DataFrame(rows)
    summary = df.agg({
        'nit':'median', 'obj':'median',
        'feas':'max', 'lb':'max', 'ub':'max',
        'wL1':'median', 'wLinf':'median'
    }).rename({'feas':'feas_max','lb':'lb_max','ub':'ub_max'})

    print("Risk-Budgeting: μ, Σ input sensitivity (medians / max violations)")
    print(summary.round(6).to_string())
    return df, summary, base

df_rb, sum_rb, base_rb = rb_input_sensitivity(
    mu_BL, Sigma_lw, budget=w_mkt,
    R=5, mu_scale=0.01, sig_scale=0.01, seed=42, c=0.025
)

---

## Results

### Allocation comparison
To compare allocation patterns across the different optimization methods, portfolio weights are displayed side by side in grouped bar charts. Each bar represents the normalized weight of an asset under a specific allocation scheme, including Equal Weight, Market Capitalisation, Mean–Variance, Maximum Sharpe ratio, and Risk Budgeting. This visualisation highlights similarities and differences in asset allocations, making it easier to identify concentration, diversification, and systematic tilts induced by each method.

In [ ]:
print(opt_weights.round(4))

In [ ]:
cols_to_plot = opt_weights.columns
labels  = ['Equal Weight','Market Cap','Mean-Variance','Max Sharpe','Risk Budgeting']
colors  = ['lightgray',   'orange',    "#0f88ce",   "#13a05e",   "#e34646"]

x = np.arange(N)
bar_width = 0.8 / len(cols_to_plot)

fig, ax = plt.subplots(figsize=(12, 6))

for i, (col, lab, colr) in enumerate(zip(cols_to_plot, labels, colors)):
    ax.bar(
        x + (i - (len(cols_to_plot)-1)/2) * bar_width,
        opt_weights[col].values,
        width  = bar_width,
        label  = lab,
        color  = colr,
    )

ax.set_xticks(x)
ax.set_xticklabels(opt_weights.index)
for xi in x[:-1]:
    ax.axvline(x=xi + 0.5, linestyle='--', linewidth=0.5,
               color='gray', alpha=0.7)

ax.set_xlim(-0.5, N - 0.5)
ax.set_ylabel('Normalized weight')
ax.set_title('Portfolio Allocations (Normalized)')
ax.legend(ncol=3)
plt.tight_layout()
plt.show()


### Weight Tilts
To better illustrate how each optimization method departs from the market portfolio, portfolio tilts are computed as the difference between the optimal weights and the corresponding market weights. These tilts are then displayed with diverging lollipop charts, where positive values indicate overweight relative to the market and negative values indicate underweight.

In [ ]:
def plot_tilts_stacked(opt_weights, w_list, colors, methods, title=None):
    """Create a stacked lollipop chart of portfolio tilts vs market weights."""
    
    tilts_df = (opt_weights[w_list].subtract(opt_weights['w_mkt'], axis=0))

    tilts_df = tilts_df.iloc[::-1] # flip to have assets listed top→bottom
    y = np.arange(len(tilts_df))
    xlim = 1.10 * tilts_df.abs().to_numpy().max()

    n = len(w_list)
    fig, axes = plt.subplots(nrows=n, ncols=1, figsize=(10, 3.2*n), sharex=True, sharey=True)

    if n == 1:
        axes = [axes]

    for ax, col, color, name in zip(axes, w_list, colors, methods):
        x1 = tilts_df[col].values
        ax.hlines(y, 0.0, x1, linewidth=2, color=color)
        ax.plot(x1, y, 'o', color=color, ms=5)
        ax.axvline(0, linestyle='--', linewidth=1, color='black')

        ax.set_yticks(y)
        ax.set_yticklabels(tilts_df.index, fontsize=8)

        ax.grid(True, axis='x', linestyle=':', alpha=0.6)
        ax.set_xlim(-xlim, xlim)
        ax.set_title(name, loc='left', fontsize=11, pad=2)

    axes[-1].set_xlabel('Tilt vs market ($w_{opt} - w_{mkt}$)', fontsize=10)
    fig.suptitle(title, y=0.99, fontsize=12)

    fig.tight_layout(pad=0.3)
    fig.subplots_adjust(top=0.93)

    plt.show()

colors  = ["#0f88ce", "#13a05e", "#e34646"]
w_list  = ['w_mv','w_ms','w_rb']
methods = ['Mean-Variance','Max Sharpe','Risk Budgeting']

plot_tilts_stacked(opt_weights, w_list, colors, methods,
                   title='Tilts vs market')

### Risk Contribution
To assess how portfolio risk is distributed across assets, risk contributions are computed as the product of portfolio weights and marginal contributions to risk, normalized by total portfolio variance. The resulting percentage contribution of each asset is displayed in bar charts for each allocation method. The dashed line indicates the equal-risk benchmark, enabling a direct comparison between the actual and the idealised equal-risk distribution. Annotated deviations highlight which assets contribute disproportionately to overall risk.

In [ ]:
def risk_contribution(weight, method):
    """Plot and report the risk contributions of a given portfolio."""
    
    w = opt_weights[weight]
    port_var    = w @ Sigma_lw @ w 
    marginal_rc = Sigma_lw @ w     
    rc          = w * marginal_rc
    pct_rc      = 100 * rc / port_var

    rc_df = pd.DataFrame({
        'Weight':               w,
        'Marginal RC':          marginal_rc,
        'Risk Contribution':    rc,
        '% Risk Contribution':  pct_rc
    }, index=weights_df.index).round(4)

    equal_rc = 100/N
    colors   = ['green' if x>equal_rc else 'red' for x in pct_rc]
    tilt_rc  = pct_rc - equal_rc

    x = np.arange(N)

    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.bar(x, rc_df['% Risk Contribution'], color=colors)

    # equal‐risk line
    ax.axhline(equal_rc, linestyle='--', color='gray', linewidth=1,
            label=f'Equal-risk ({equal_rc:.2f}%)')

    for bar, d in zip(bars, tilt_rc):
        y = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, 
                y + 0.5, 
                f"{d:+.2f}%", 
                ha='center', va='bottom', fontsize=9)

    ax.set_xticks(x)
    ax.set_xticklabels(rc_df.index, rotation=45, ha='right')

    ax.set_ylabel('% Risk Contribution')
    ax.set_title(f'Risk Contributions of {method} Portfolio')
    ax.legend()
    plt.tight_layout()
    plt.show()

for (weight, method) in zip(w_list, methods):
    risk_contribution(weight, method)

### Performance Statistics
To compare the efficiency of the different allocation schemes, ex-ante performance measures are computed. For each portfolio, the following indicators are reported:
- Expected return ($\mu$), annualized and expressed in percent.
- Variance ($\sigma^2$), annualized and expressed in percent.
- Volatility ($\sigma$), as the square root of variance (annualized).
- Sharpe ratio, defined as expected return per unit of volatility (annualized).
- Diversification ratio (DR), computed as the weighted average of individual asset volatilities divided by the portfolio volatility.

The resulting table allows a systematic comparison of the risk–return trade-off and diversification properties across all portfolio specifications.

In [ ]:
def compute_stats(weight_col, mu_BL, Sigma, opt_weights):
    """Compute performance statistics for given weight:
    mean, variance, volatility and diversification ratio"""
    w   = opt_weights[weight_col].values       
    mu  = float(w @ mu_BL)
    var = float(w @ Sigma @ w)
    vol = np.sqrt(var)         

    # Sharpe ratio 
    sharpe = mu / vol if vol > 0 else np.nan

    # Diversification Ratio 
    sigma_assets = np.sqrt(np.diag(Sigma))         
    dr = (w @ sigma_assets) / vol if vol > 0 else np.nan

    return {"mu [% p.a.]": (mu * WEEKS_PER_YEAR) * 100, 
            "var [% p.a.]": (var * WEEKS_PER_YEAR) * 100, 
            "vol (p.a.)": (vol * np.sqrt(WEEKS_PER_YEAR)), 
            "sharpe (p.a.)": (sharpe * np.sqrt(WEEKS_PER_YEAR)), 
            "DR": dr
            }

w_list = opt_weights.columns

results = {col: compute_stats(col, mu_BL, Sigma_lw, opt_weights) for col in w_list}
results_df = pd.DataFrame(results).T
results_df

### Portfolio Exposure
To assess the sources of portfolio risk and return, each set of optimised weights is regressed on a set of factor returns (Fama–French five factors plus Momentum and $\text{BAB}$). The regression is estimated with Newey–West heteroskedasticity and autocorrelation consistent (HAC) standard errors to account for weekly data.
For each portfolio, the following outputs are reported:
- $\alpha$: the intercept of the regression, interpreted as abnormal return unexplained by the factors.
- $\beta$: the estimated coefficients on each factor, representing systematic sensitivities.
- $R^2$: the proportion of portfolio variance explained by the factor model.

This analysis provides a factor-based decomposition of the Mean–Variance, Maximum Sharpe, and Risk Budgeting portfolios, highlighting their tilts relative to well-established systematic risk premia.

In [ ]:
def portfolio_exposures(w, R_assets, F_factors, nw_lags=4):
    """Estimate a portfolio's factor exposures via OLS with Newey-West (HAC) errors.
    Output: alpha, beta, r2."""

    R, F = R_assets.align(F_factors, join="inner", axis=0)
    r_p  = R.values @ w
    X    = sm.add_constant(F.values)
    ols  = sm.OLS(r_p, X).fit()
    nw   = ols.get_robustcov_results(cov_type="HAC", maxlags=nw_lags)
    alpha = nw.params[0]
    beta = pd.Series(nw.params[1:], index=F.columns)
    return alpha, beta, ols.rsquared

fcols = ["MKT","SMB","HML","RMW","CMA","MOM","BAB"] 
F = factors_exc[fcols].dropna()

alpha_mv, beta_mv, r2_mv = portfolio_exposures(opt_weights['w_mv'], asset_universe_exc, F)
alpha_ms, beta_ms, r2_ms = portfolio_exposures(opt_weights['w_ms'], asset_universe_exc, F)
alpha_rb, beta_rb, r2_rb = portfolio_exposures(opt_weights['w_rb'], asset_universe_exc, F)


results = {
    'MV': {'alpha (% p.a.)': 100*alpha_mv*WEEKS_PER_YEAR,
           'R2': r2_mv, **beta_mv.to_dict()},
    'MS': {'alpha (% p.a.)': 100*alpha_ms*WEEKS_PER_YEAR,
           'R2': r2_ms, **beta_ms.to_dict()},
    'RB': {'alpha (% p.a.)': 100*alpha_rb*WEEKS_PER_YEAR,
           'R2': r2_rb, **beta_rb.to_dict()},
}

df_exposures = pd.DataFrame(results).T  
df_exposures.index.name = 'Portfolio'
df_exposures = df_exposures[['alpha (% p.a.)'] + fcols + ['R2']]

print(df_exposures.round(4))